# PD Model Estimation

In [1]:
%load_ext autoreload
%autoreload 2

## Change project directory (google colab)

In [2]:
import os
os.chdir('/content/drive/MyDrive/projects/credit-risk-modelling-udemy/notebooks/')
os.listdir('../')

['venv-hpx360-win',
 'data',
 'src',
 '.git',
 'requirements.txt',
 'README.md',
 'src.egg-info',
 'setup.py',
 'notebooks',
 'reports',
 'outputs',
 '.gitignore']

In [3]:
!pip install -e ../.

Obtaining file:///content/drive/MyDrive/projects/credit-risk-modelling-udemy
  Running setup.py develop for src


## Some imports

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import src
from src.project.config import PROJECT_DIR
from IPython.display import display

## Load Data

In [4]:
fpaths = {subset: os.path.join(PROJECT_DIR, 'outputs', f'loan_data_2007_2014_{subset}_prepared_for_pd_model_nb3.pkl') for subset in ['train', 'test']}
df_train = pd.read_pickle(fpaths['train'])
df_test = pd.read_pickle(fpaths['test'])


In [5]:
df_train.head(2)

,index,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,...,int_rate:<=9.548,int_rate:9.548-12.025,int_rate:12.025-15.327,int_rate:15.327-19.868,int_rate:>=19.868,funded_amnt_binned,annual_inc_binned,annual_inc_combined,annual_inc:<=28000,annual_inc:28000-45000,annual_inc:45000-49000,annual_inc:49000-62983,annual_inc:62983-67000,annual_inc:67000-81000,annual_inc:81000-105000,annual_inc:>=105000,mths_since_last_delinq_binned,mths_since_last_delinq_combined,mths_since_last_delinq:missing,mths_since_last_delinq:0-4,mths_since_last_delinq:4-31,mths_since_last_delinq:31-57,mths_since_last_delinq:>=57,dti_binned,dti_combined,dti:<=7.598,dti:7.598-11.997,dti:11.997-16.796,dti:16.796-21.595,dti:21.595-25.994,dti:>=25.994,mths_since_last_record_binned,mths_since_last_record_combined,mths_since_last_record:missing,mths_since_last_record:0-2,mths_since_last_record:2-21,mths_since_last_record:21-32,mths_since_last_record:32-81,mths_since_last_record:81-87,mths_since_last_record:>=87
427211,427211,12796369,14818505,24000,24000,24000.0,36 months,8.90,762.08,A,A5,Supervisor inventory management,3 years,MORTGAGE,77000.0,Source Verified,2014-03-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 03/12/14 > I have 5 credit c...,debt_consolidation,Debt consolidation,295xx,SC,21.91,0.0,1986-12-01,1.0,NaN,NaN,20.0,0.0,30489,53.5,32.0,f,10098.3,10098.3,16765.76000,...,1,0,0,0,0,"(23960.0, 24650.0]","(75000.0, 81000.0]",67000-81000,0,0,0,0,0,1,0,0,NaN,missing,1,0,0,0,0,"(21.595, 21.995]",21.595-25.994,0,0,0,0,1,0,NaN,missing,1,0,0,0,0,0,0
206088,206088,1439740,1691948,10000,10000,10000.0,36 months,14.33,343.39,C,C1,mizuho corporate bank,6 years,MORTGAGE,112000.0,Not Verified,2012-08-01,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 07/23/12 > I was looking for...,debt_consolidation,Credit card consolidation,070xx,NJ,7.49,1.0,1997-12-01,2.0,18.0,NaN,15.0,0.0,15836,53.1,38.0,f,0.0,0.0,12357.02066,...,0,0,1,0,0,"(9470.0, 10160.0]","(105000.0, 120000.0]",>=105000,0,0,0,0,0,0,0,1,"(15.04, 18.8]",4-31,0,0,1,0,0,"(7.198, 7.598]",<=7.598,1,0,0,0,0,0,NaN,missing,1,0,0,0,0,0,0


In [6]:
df_test.head(2)

,index,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,...,int_rate:<=9.548,int_rate:9.548-12.025,int_rate:12.025-15.327,int_rate:15.327-19.868,int_rate:>=19.868,funded_amnt_binned,annual_inc_binned,annual_inc_combined,annual_inc:<=28000,annual_inc:28000-45000,annual_inc:45000-49000,annual_inc:49000-62983,annual_inc:62983-67000,annual_inc:67000-81000,annual_inc:81000-105000,annual_inc:>=105000,mths_since_last_delinq_binned,mths_since_last_delinq_combined,mths_since_last_delinq:missing,mths_since_last_delinq:0-4,mths_since_last_delinq:4-31,mths_since_last_delinq:31-57,mths_since_last_delinq:>=57,dti_binned,dti_combined,dti:<=7.598,dti:7.598-11.997,dti:11.997-16.796,dti:16.796-21.595,dti:21.595-25.994,dti:>=25.994,mths_since_last_record_binned,mths_since_last_record_combined,mths_since_last_record:missing,mths_since_last_record:0-2,mths_since_last_record:2-21,mths_since_last_record:21-32,mths_since_last_record:32-81,mths_since_last_record:81-87,mths_since_last_record:>=87
362514,362514,19677589,21900299,32500,32500,32500.0,60 months,14.99,773.01,C,C5,Licensed vocational nurse,< 1 year,MORTGAGE,65000.0,Verified,2014-07-01,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,home_improvement,Home improvement,950xx,CA,9.32,0.0,2002-04-01,0.0,NaN,NaN,7.0,0.0,1208,4.3,20.0,f,0.00,0.00,38213.56,...,0,0,1,0,0,"(32240.0, 32930.0]","(63000.0, 67000.0]",62983-67000,0,0,0,0,1,0,0,0,NaN,missing,1,0,0,0,0,"(9.198, 9.598]",7.598-11.997,0,1,0,0,0,0,NaN,missing,1,0,0,0,0,0,0
288564,288564,29755527,32278795,11000,11000,11000.0,60 months,20.99,297.53,E,E4,shift manager,10+ years,MORTGAGE,60000.0,Verified,2014-10-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,credit_card,Credit card refinancing,472xx,IN,24.19,1.0,2003-08-01,0.0,3.0,NaN,18.0,0.0,13129,54.7,43.0,w,9214.67,9214.67,4450.12,...,0,0,0,0,1,"(10850.0, 11540.0]","(60000.0, 63000.0]",49000-62983,0,0,0,1,0,0,0,0,"(-0.17, 3.4]",missing,1,0,0,0,0,"(23.994, 24.394]",21.595-25.994,0,0,0,0,1,0,NaN,missing,1,0,0,0,0,0,0


## Load Dummy Variable List

In [7]:
infile = os.path.join(PROJECT_DIR, 'outputs', 'dummy_var_lst.csv')
dummy_var_df = pd.read_csv(infile)
dummy_var_df.head()

,dummy_var
0,grade:G
1,grade:F
2,grade:E
3,grade:D
4,grade:C


In [8]:
dummy_var_df['var'] = dummy_var_df['dummy_var'].str.extract(f'(?P<var>.+):.')
dummy_var_df


,dummy_var,var
0,grade:G,grade
1,grade:F,grade
2,grade:E,grade
3,grade:D,grade
4,grade:C,grade
...,...,...
62,mths_since_last_record:2-21,mths_since_last_record
63,mths_since_last_record:21-32,mths_since_last_record
64,mths_since_last_record:32-81,mths_since_last_record
65,mths_since_last_record:81-87,mths_since_last_record


In [9]:
dummy_var_df.loc[dummy_var_df['var'] == 'mths_since_last_record']

,dummy_var,var
60,mths_since_last_record:missing,mths_since_last_record
61,mths_since_last_record:0-2,mths_since_last_record
62,mths_since_last_record:2-21,mths_since_last_record
63,mths_since_last_record:21-32,mths_since_last_record
64,mths_since_last_record:32-81,mths_since_last_record
65,mths_since_last_record:81-87,mths_since_last_record
66,mths_since_last_record:>=87,mths_since_last_record


In [10]:
REFERENCE_CATEGORIES = dummy_var_df.drop_duplicates('var').reset_index(drop=True).set_index('var')
REFERENCE_CATEGORIES.loc['mths_since_last_delinq', 'dummy_var'] = 'mths_since_last_delinq:0-4'
REFERENCE_CATEGORIES.loc['mths_since_last_record', 'dummy_var'] = 'mths_since_last_record:0-2'
REFERENCE_CATEGORIES = REFERENCE_CATEGORIES['dummy_var'].tolist()
REFERENCE_CATEGORIES

['grade:G',
 'home_ownership:OTHER__NONE__ANY__RENT',
 'verification_status:Verified',
 'purpose:small_business__educational__moving__house',
 'initial_list_status:f',
 'term_int:36',
 'emp_length_int:0',
 'mths_since_issue_d:<=38',
 'int_rate:<=9.548',
 'annual_inc:<=28000',
 'mths_since_last_delinq:0-4',
 'dti:<=7.598',
 'mths_since_last_record:0-2']

In [11]:
DUMMY_VARIABLES = pd.Index(dummy_var_df['dummy_var']).difference(REFERENCE_CATEGORIES)
DUMMY_VARIABLES

Index(['annual_inc:28000-45000', 'annual_inc:45000-49000',
       'annual_inc:49000-62983', 'annual_inc:62983-67000',
       'annual_inc:67000-81000', 'annual_inc:81000-105000',
       'annual_inc:>=105000', 'dti:11.997-16.796', 'dti:16.796-21.595',
       'dti:21.595-25.994', 'dti:7.598-11.997', 'dti:>=25.994',
       'emp_length_int:1', 'emp_length_int:10', 'emp_length_int:2-4',
       'emp_length_int:5-6', 'emp_length_int:7-9', 'grade:A', 'grade:B',
       'grade:C', 'grade:D', 'grade:E', 'grade:F', 'home_ownership:MORTGAGE',
       'home_ownership:OWN', 'initial_list_status:w', 'int_rate:12.025-15.327',
       'int_rate:15.327-19.868', 'int_rate:9.548-12.025', 'int_rate:>=19.868',
       'mths_since_issue_d:38-40', 'mths_since_issue_d:40-42',
       'mths_since_issue_d:42-49', 'mths_since_issue_d:49-53',
       'mths_since_issue_d:53-65', 'mths_since_issue_d:65-85',
       'mths_since_issue_d:>=85', 'mths_since_last_delinq:31-57',
       'mths_since_last_delinq:4-31', 'mths_since_l

In [12]:
import json
d = {'DUMMY_VARIABLES': list(DUMMY_VARIABLES), 
     'REFERENCE_CATEGORIES': list(REFERENCE_CATEGORIES)}

out_file = os.path.join(PROJECT_DIR, 'outputs', 'dummy_vars_ref_cats.json')
with open(out_file, 'w') as f:
    json.dump(d, f)

## Prepare $X$ and $y$

In [13]:
X_train, y_train = df_train[DUMMY_VARIABLES], df_train['good_bad_loan'].to_numpy().astype(int)
X_test, y_test = df_test[DUMMY_VARIABLES], df_test['good_bad_loan'].to_numpy().astype(int)

In [14]:
print('Train:', X_train.shape, 'Test:', X_test.shape)

Train: (373028, 54) Test: (93257, 54)


## Probability of Default Modelling using `LogisticRegression`

In [15]:
from sklearn.linear_model import LogisticRegression
import scipy

In [16]:
for varname in LogisticRegression.__init__.__code__.co_varnames:
    print(f'self.{varname} = {varname}')
    


self.self = self
self.penalty = penalty
self.dual = dual
self.tol = tol
self.C = C
self.fit_intercept = fit_intercept
self.intercept_scaling = intercept_scaling
self.class_weight = class_weight
self.random_state = random_state
self.solver = solver
self.max_iter = max_iter
self.multi_class = multi_class
self.verbose = verbose
self.warm_start = warm_start
self.n_jobs = n_jobs
self.l1_ratio = l1_ratio


In [19]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import scipy

class LogisticRegressionWithPValues(LogisticRegression):

    def fit(self, X, y):
        super().fit(X, y)

        denom = (2.0 * (1.0 + np.cosh(self.decision_function(X))))
        denom = np.tile(denom,(X.shape[1],1)).T
        F_ij = np.dot((X / denom).T,X) ## Fisher Information Matrix
        Cramer_Rao = np.linalg.inv(F_ij) ## Inverse Information Matrix
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        z_scores = self.coef_[0] / sigma_estimates # z-score for eaach model coefficient
        p_values = [scipy.stats.norm.sf(abs(x)) * 2 for x in z_scores] ### two tailed test for p-values

        self.p_values = p_values
        try:
            self.feature_names = X.columns
        except AttributeError:
            self.feature_names = [f'X{i}' for i in range(X.shape[1])]

        return self

    def get_summary_table(self, feature_names=None):

        if feature_names is None:
            feature_names = self.feature_names

        # Same as above.
        summary_table = pd.DataFrame(columns = ['feature_name'], data = feature_names)
        summary_table['coefficient'] = np.transpose(self.coef_)
        summary_table.index = summary_table.index + 1
        summary_table.loc[0] = ['intercept', self.intercept_[0]]
        summary_table = summary_table.sort_index()
        summary_table['p_value'] = np.append(np.nan, np.array(self.p_values))
        

        return summary_table

In [20]:
clf = LogisticRegressionWithPValues(solver='saga')
clf.fit(X_train, y_train)

LogisticRegressionWithPValues(C=1.0, class_weight=None, dual=False,
                              fit_intercept=True, intercept_scaling=1,
                              l1_ratio=None, max_iter=100, multi_class='auto',
                              n_jobs=None, penalty='l2', random_state=None,
                              solver='saga', tol=0.0001, verbose=0,
                              warm_start=False)

In [21]:
smry_table = clf.get_summary_table()
smry_table

,feature_name,coefficient,p_value
0,intercept,1.319336,NaN
1,annual_inc:28000-45000,0.067684,3.784238e-03
2,annual_inc:45000-49000,0.151736,1.635146e-07
3,annual_inc:49000-62983,0.205341,1.209586e-17
4,annual_inc:62983-67000,0.254086,2.059925e-15
5,annual_inc:67000-81000,0.339545,1.383467e-39
6,annual_inc:81000-105000,0.439966,2.797638e-61
7,annual_inc:>=105000,0.547955,4.412025e-90
8,dti:11.997-16.796,-0.129152,8.665769e-10
9,dti:16.796-21.595,-0.228966,1.098879e-27


## Write out summary table

In [23]:
out_file = os.path.join(PROJECT_DIR, 'outputs', 'pd_model_logreg_summary_table.csv')
smry_table.to_csv(out_file, index=False)

## Write out Logistic Regression model

In [24]:
import pickle

In [25]:
out_file = os.path.join(PROJECT_DIR, 'outputs', 'pd_model_logreg.pkl')
with open(out_file, 'wb') as f:
    pickle.dump(clf, f)